In [ ]:
import numpy as np
import pandas as pd
import pyNetLogo
import os
import ipyparallel
from os import environ

# Start IPCluster with 8 engines now through the command line using the following command:
# IPCluster start -n 8
# 8 engines are used for fair comparison against 8 parallel headless workspaces through NL4Py
client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

In [ ]:
%%px 
#Jupyter notebook decorator to make the notebook cell parallelizable
import os
os.chdir(cwd) # change to the current directory to the IPCluster engines
import pyNetLogo
import pandas as pd
if os.environ.get('netlogo_path') is not None:
    netlogo_path = os.environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"
print(netlogo_path)
# Replace argument netlogo_home with the location of your NetLogo installation 
netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home = netlogo_path, netlogo_version = '6')
netlogo.load_model('../Models/Wolf Sheep Predation.nlogo')

In [ ]:
def simulation(runId):
    try:
        # Same netlogo commands as used for the NL4Py evaluation
        netlogo.command("random-seed 999")
        netlogo.command("set initial-number-sheep 150")
        netlogo.command("set initial-number-wolves 150")
        netlogo.command("set grass-regrowth-time 50")
        netlogo.command("set sheep-gain-from-food 25")
        netlogo.command("set wolf-gain-from-food 50")
        netlogo.command("set sheep-reproduce 10")
        netlogo.command("set wolf-reproduce 10")
        netlogo.command("set show-energy? false")
        netlogo.command('set model-version "sheep-wolves-grass"')
        netlogo.command('setup')
        #Run for 100 ticks and return the number of sheep and wolf agents at each time step
        counts = netlogo.repeat_report(['ticks','count sheep','count wolves'], 100)    
        print(counts)
        results = pd.Series([counts.shape[0],
                             counts.iloc[-1:1], 
                             counts.iloc[-1:2]],index=['ticks','Avg. sheep', 'Avg. wolves'])
        print(results)
        return results
    except:
        pass    

In [ ]:
import os
outputFile = '../output/5.3_output.csv'
out = open (outputFile, "a+")
print("ready")

In [ ]:
import time
for runs in [5000, 10000, 15000]:
    startTime = int(round(time.time() * 1000))
    # Run the simulations and measure execution time
    try:
        #The following failsafe had to be added as PyNetLogo fails due to duplicated temporary log file names that are left behind from previous runs
        test = os.listdir(os.getcwd())
        for item in test:
            if item.endswith(".txt"):
                os.remove(os.path.join(os.getcwd(), item))
        ## end failsafe
        ## load balance view failed to run consistently. Therefore, reverted to using direct view in order to get PyNetLogo running on an IPCluster
        results = direct_view.map_sync(simulation, range(runs))
    except Exception as e: 
        pass
    stopTime = int(round(time.time() * 1000))
    executionTime = stopTime - startTime
    print("pyNetLogo,repeatReporters," + str(runs) + "," + str(executionTime))
    out.write("pyNetLogo,repeatReporters," + str(runs) + "," + str(executionTime) + "\n")
    out.flush()

In [ ]:
out.close()